In [9]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Neo4jVector
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
import streamlit as st
import tempfile
from neo4j import GraphDatabase

load_dotenv()

True

In [10]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-4o")

In [11]:
neo4j_uri = os.getenv("NEO4J_URI")
neo4j_username = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")

In [ ]:
graph = Neo4jGraph(url=neo4j_uri, username=neo4j_username, password=neo4j_password)

In [ ]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

In [13]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-haiku-20241022", temperature=0.1, max_retries=2, max_tokens=1024)

In [14]:
llm.invoke("hi")

AIMessage(content='Hello! How are you doing today? Is there anything I can help you with?', additional_kwargs={}, response_metadata={'id': 'msg_017rSgBqY6EafxRYPpfiRPXK', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 8, 'output_tokens': 20, 'server_tool_use': None, 'service_tier': 'standard', 'cache_creation': {'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run--58c4a38c-5f3e-4d0c-be77-aed8ca25b2a5-0', usage_metadata={'input_tokens': 8, 'output_tokens': 20, 'total_tokens': 28, 'input_token_details': {'cache_read': 0, 'cache_creation': 0, 'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}})

In [15]:
messages = [
    (
        "system", 
        "You are helpful assistant that translates ENglish to Urdu. Translate the user sentence."
    ),
    ("human", "I love programming")
]

In [16]:
ai_msg = llm.invoke(messages)

In [17]:
ai_msg

AIMessage(content='میں پروگرامنگ سے محبت کرتا ہوں۔', additional_kwargs={}, response_metadata={'id': 'msg_01E8kS96KbGrZYE78u4Fmiym', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 31, 'output_tokens': 28, 'server_tool_use': None, 'service_tier': 'standard', 'cache_creation': {'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run--d0d7767a-78bc-426f-a3aa-1706d3b05266-0', usage_metadata={'input_tokens': 31, 'output_tokens': 28, 'total_tokens': 59, 'input_token_details': {'cache_read': 0, 'cache_creation': 0, 'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
[
    (
        "system", 
        "You are helpful assistant that translates {input_language} to {output_language}. Translate the user sentence."
    ),
    ("human", "{input}")
]
)

chain = prompt | llm

In [19]:
response = chain.invoke({"input_language": "English", "output_language": "Urdu", "input": "The capital of Pakistan is Islamabad"})

In [20]:
response.content

'پاکستان کا دارالحکومت اسلام آباد ہے۔'

In [21]:
from pydantic import BaseModel, Field

class GetWeather(BaseModel):
    """Get the current weather in a given location"""
    
    location: str = Field(default="", description="The city and state, e.g Karachi, Islamabad")

In [22]:
llm_with_tools = llm.bind_tools([GetWeather])

In [23]:
ai_msg = llm_with_tools.invoke("Which city is hotter today Karachi or Islamabad?")

In [24]:
ai_msg

AIMessage(content=[{'text': "I'll help you compare the temperatures of Karachi and Islamabad by checking their current weather. I'll retrieve the weather information for both cities.", 'type': 'text'}, {'id': 'toolu_01NYuFhRotgReEjpM2UwAjoA', 'input': {'location': 'Karachi, Pakistan'}, 'name': 'GetWeather', 'type': 'tool_use'}, {'id': 'toolu_01MmuvUg2HPHXaD1DTmPUtxy', 'input': {'location': 'Islamabad, Pakistan'}, 'name': 'GetWeather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01Uzn5fP47GwngTzxDqk3dEp', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 359, 'output_tokens': 129, 'server_tool_use': None, 'service_tier': 'standard', 'cache_creation': {'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run--4a57f629-7a0c-497e-b57a-8697daa0ccb5-0', tool_calls=[{'name': 'G

In [26]:
import anthropic

client = anthropic.Anthropic()
file = client.beta.files.upload(file=open("test_image.jpg", "rb"))

image_file_id = file.id

In [27]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-haiku-20241022", betas=["files-api-2025-04-14"])

In [28]:
input_message = {
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Describe this image",
        },
        {
            "type": "image",
            "source": {
                "type":"file",
                "file_id": image_file_id
            }
        }
    ]
}

In [29]:
llm.invoke([input_message])

AIMessage(content='This image captures a dramatic wildlife scene on the African savanna. A hyena is confronting or challenging a gazelle or antelope, while a cheetah is running in the background, apparently in pursuit. The scene illustrates the intense predator-prey dynamics of the grassland ecosystem.\n\nThe hyena appears to be in a threatening posture, with its body low and confrontational. The antelope seems to be mid-leap, trying to escape the potential danger. The cheetah, known for its incredible speed, is running in the background, adding another layer of tension to the scene.\n\nThe landscape is a typical savanna setting, with a grassy, slightly brownish-green terrain. The animals are positioned in a way that creates a sense of motion and imminent action, showcasing the constant struggle for survival in the wild.\n\nThis image provides a raw, unfiltered look at the harsh realities of life in the African wilderness, where predators and prey are constantly engaged in a life-or-de